## 商品型（抽象型）

In [1]:
abstract type 商品 end

In [2]:
本体価格(v::商品) = v.price

本体価格 (generic function with 1 method)

## 税率型（抽象型、Trait型）

In [3]:
abstract type 税率 end

In [4]:
struct 標準税率 <: 税率 end
struct 軽減税率 <: 税率 end

In [5]:
税込価格(v::T) where {T <: 商品} = 税込価格(税率(T), v)
税込価格(::標準税率, v) = floor(Int, 本体価格(v) * 1.1)
税込価格(::軽減税率, v) = floor(Int, 本体価格(v) * 1.08)

税込価格 (generic function with 3 methods)

### 標準税率

In [6]:
税率(::Type{<:商品}) = 標準税率()

税率

#### 例

In [7]:
struct 衣料品 <: 商品
    name::String
    price::Int
end

In [8]:
コート = 衣料品("ダウンコート", 30000)

衣料品("ダウンコート", 30000)

In [9]:
税込価格(コート)

33000

In [10]:
struct 書籍 <: 商品
    name::String
    author::String
    price::Int
end

In [11]:
本₁ = 書籍("龍が目を覚ます", "宮木みゆべ", 1980)

書籍("龍が目を覚ます", "宮木みゆべ", 1980)

In [12]:
税込価格(本₁)

2178

### 軽減税率

#### 新聞（定期購読）

In [13]:
using Dates

In [14]:
struct 定期購読新聞 <: 商品
    name::String
    term::AbstractRange{Date}
    price::Int
end

定期購読新聞(name, date::Date, price) = 定期購読新聞(name, date:Day(1):lastdayofmonth(date), price)
定期購読新聞(name, startdate::Date, enddate::Date, price) = 定期購読新聞(name, startdate:Day(1):enddate, price)

定期購読新聞

In [15]:
税率(::Type{定期購読新聞}) = 軽減税率()

税率

↑Point！

In [16]:
新聞_年間契約 = 定期購読新聞("毎朝新聞", Date(2020, 2, 1), Date(2021, 1, 31), 50000)

定期購読新聞("毎朝新聞", 2020-02-01:1 day:2021-01-31, 50000)

In [17]:
税込価格(新聞_年間契約)

54000

#### 飲食料品…

In [18]:
struct 飲食料品 <: 商品
    name::String
    price::Int
end

In [ ]:
# 税率(::Type{飲食料品}) = 軽減税率()  # ???

#### 飲食料品： 持ち帰りか店内飲食か

In [19]:
abstract type 提供形態 end
struct 持ち帰り <: 提供形態 end
struct 店内飲食 <: 提供形態 end

In [20]:
税率(::Type{飲食料品}, ::Type{<:提供形態}) = 標準税率()
税率(::Type{飲食料品}, ::Type{持ち帰り}) = 軽減税率()

税率

↑Point！

In [21]:
税込価格(v::飲食料品, ::K = 持ち帰り()) where {K <: 提供形態} = 税込価格(税率(飲食料品, K), v)

税込価格 (generic function with 5 methods)

In [22]:
外食() = 店内飲食()
イートイン() = 店内飲食()

出前() = 持ち帰り()
宅配() = 持ち帰り()

宅配 (generic function with 1 method)

In [23]:
あんぱん = 飲食料品("あんぱん", 128)

飲食料品("あんぱん", 128)

In [24]:
税込価格(あんぱん)

138

In [25]:
税込価格(あんぱん, 持ち帰り())

138

In [26]:
税込価格(あんぱん, イートイン())

140

In [27]:
そば = 飲食料品("かけそば", 580)

飲食料品("かけそば", 580)

In [28]:
税込価格(そば, 店内飲食())

638

In [29]:
税込価格(そば, 出前())

626

Memo:

飲食料品は、商品そのものの性質以外に、提供形態（＝購入時の状況）によって税率が変わる  
  → 税込価格を計算する際に、商品以外に「提供形態」の情報が必要  
  → 2引数

#### 一体型商品…

飲食料品とその他（おまけ、容器等）の一体型商品の場合：

+ 飲食料品以外が価格比で本体価格の 1/3 を超える → 軽減税率対象外（＝標準税率）
    + 例：ティーカップとお菓子のセット、おまけがメインの食玩・駄菓子
+ 本体価格が ¥10,000 以上 → 軽減税率対象外（＝標準税率）
    + 例：高級おせち料理セット
+ 上記2つをいずれも満たさないもの → 軽減税率
    + 例：おまけシール付きのお菓子（おまけのほうが十分に安いもの）

In [30]:
abstract type PriceCost end
struct Expensive <: PriceCost end
struct Inexpensive <: PriceCost end

PriceCost(v::Int) = v ≥ 10000 ? Expensive() : Inexpensive()

PriceCost

In [31]:
abstract type PriceRateOfFood end
struct HighRate <: PriceRateOfFood end
struct LowRate <: PriceRateOfFood end

PriceRateOfFood(price, groceriesprice) = (price - groceriesprice) * 3 > price ? LowRate() : HighRate()

PriceRateOfFood

In [32]:
struct 一体型商品{PC<:PriceCost, PR<:PriceRateOfFood} <: 商品
    name::String
    price::Int
    function 一体型商品(name, price, groceriesprice)
        pc = PriceCost(price)
        pr = PriceRateOfFood(price, groceriesprice)
        new{typeof(pc), typeof(pr)}(name, price)
    end
end

In [33]:
税率(::Type{<:一体型商品}) = 標準税率()
税率(::Type{一体型商品{Inexpensive, HighRate}}) = 軽減税率()

税率

↑Point！

In [34]:
おせちセット = 一体型商品("高級おせち料理セット", 12000, 8000)
税込価格(おせちセット)  # 標準税率

13200

In [35]:
チョコたまご = 一体型商品("チョコたまご", 400, 200)
税込価格(チョコたまご)  # 標準税率

440

In [36]:
シールチョコ = 一体型商品("おまけシール付きチョコ", 120, 100)
税込価格(シールチョコ)  # 軽減税率

129

### 応用：商品購入型

In [37]:
struct 商品購入{G <: 商品, K <: 提供形態}
    商品::G
    購入形態::K
    商品購入(v::G, k::K) where {G <: 商品, K <: 提供形態} = new{G, K}(v, k)
end

商品購入(v::商品) = 商品購入(v, 持ち帰り())

商品購入

In [42]:
税込価格(p::P) where {P <: 商品購入} = 税込価格(税率(P), p.商品)

税込価格 (generic function with 6 methods)

#### 標準税率

In [39]:
税率(::Type{<:商品購入}) = 標準税率()

税率

In [40]:
購入₁ = 商品購入(本₁) 

商品購入{書籍,持ち帰り}(書籍("龍が目を覚ます", "宮木みゆべ", 1980), 持ち帰り())

In [43]:
税込価格(購入₁)

2178

#### 標準税率 or 軽減税率

In [44]:
# 税率(::Type{<:商品購入{飲食料品}}) = 標準税率()  # ← なくてもOK
税率(::Type{商品購入{飲食料品, 持ち帰り}}) = 軽減税率()  # ←こっちが重要

税率

In [45]:
あんぱん_持ち帰り = 商品購入(あんぱん, 持ち帰り())

商品購入{飲食料品,持ち帰り}(飲食料品("あんぱん", 128), 持ち帰り())

In [46]:
税込価格(あんぱん_持ち帰り)

138

In [47]:
あんぱん_イートイン = 商品購入(あんぱん, イートイン())

商品購入{飲食料品,店内飲食}(飲食料品("あんぱん", 128), 店内飲食())

In [48]:
税込価格(あんぱん_イートイン)

140

In [49]:
# 税率(::Type{<:商品購入{<:一体型商品}}) = 標準税率()  # ← なくてもOK
税率(::Type{商品購入{一体型商品{Inexpensive, HighRate}, 持ち帰り}}) = 軽減税率()  # ←こっちが重要

税率

In [50]:
税込価格(商品購入(チョコたまご, 持ち帰り()))

440

In [51]:
税込価格(商品購入(シールチョコ, 持ち帰り()))

129

In [52]:
税込価格(商品購入(シールチョコ, イートイン()))

132